# PhoneRecommender

### *Autheurs : Baudet Quentin et Larmaillard-Noiren Joris*


PhoneRecommender est un chatbot de recommandation de smartphones basé sur un pipeline **Retrieval-Augmented Generation (RAG)** et un modèle de langage pré-entraîné.

---

## Contexte et objectifs

Ce projet a pour but de construire un assistant intelligent capable d’aider un utilisateur à choisir un téléphone portable en fonction de critères (prix, performance, autonomie, photo, etc.).
Pour cela, nous exploitons :
- Un **dataset Kaggle** contenant les fiches techniques et les spécifications de centaines de modèles de smartphones.
- Un pipeline **RAG** : on indexe les descriptions produits dans un moteur de similarité (FAISS) puis on combine la récupération de documents pertinents à la génération de texte par un **LLM** (Mistral 7B, GPT-2, LLaMA …, au choix).

---

## Fonctionnalités

1. **Exploration et nettoyage du dataset**
   - Analyse exploratoire (EDAx).
   - Standardisation des noms de champs (RAM, stockage, autonomie…).
2. **Indexation documentaire**
   - Conversion du CSV en fragments textuels.
   - Construction d’un index FAISS pour recherches rapides.
3. **Pipeline RAG**
   - À chaque question utilisateur, récupération des **k fiches les plus pertinentes**.
   - Enrichissement du prompt avec ces extraits.
   - Génération de la réponse via un LLM.
4. **Questions réponses dans le terminal**
   - L'utilisateur obtient sa recommandation de téléphone dans le terminal

 
---

## Répartition du travail

Joris Larmaillard-Noiren s'est penché sur l'exploration du dataset et la mise en place de l'architecture du modèle RAG et ses vérifications.
Quentin Baudet s'est occupé du preprocessing des données et de la mise en place du modèle et l'interprétation des résultats du modèle.

Les étapes ayant pris le plus de temps ont été l'EDA/traitement (essentiel pour donner sens et enrichir les données) et la mise en place de la pipeline (nombreux conflits de versions de packages).

## Exploration du dataset

Avant de construire notre modèle de recommandation de smartphones, il est crucial de bien comprendre les caractéristiques et la structure du dataset fourni. Dans ce notebook, nous réalisons une Analyse Exploratoire des Données (EDA) afin de mieux cerner la distribution des différentes variables, détecter les éventuelles anomalies, valeurs manquantes ou incohérences, et identifier les relations potentielles entre les attributs.

Cette étape nous permettra d’extraire des insights précieux pour orienter le pré-traitement des données et la sélection des fonctionnalités les plus pertinentes, en vue d’optimiser les performances du modèle de recommandation.

### Chargement des données

In [1]:
### Importation des modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
### Chargement des données
data = pd.read_csv('../data/raw/Smartphones_cleaned_dataset_completed.csv', sep=";")

data.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,38999,73.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),46999,75.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,109900,77.0,False,True,False,bionic,6,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,51999,74.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),55999,75.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532


Dans un premier temps nous allons reformater les prix en euros : les données qui sont présentées ci-dessus viennent du marché des smartphones en Inde. Nous allons donc faire une conversion des prix, ici en roupie, en euros.

In [3]:
### Conversion des prix roupie -> euro
data['price'] = data['price'] * 0.009966

### Arrondi des prix
data['price'] = data['price'].round(2)

data = data.rename(columns={'price': 'price (€)'})

data.head()

,brand_name,model,price (€),rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,388.66,73.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,True,False,bionic,6,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,518.22,74.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532


**Dimension du dataset**

In [4]:
data.shape

(979, 26)

**Colonnes du dataset**

In [5]:
data.columns

Index(['brand_name', 'model', 'price (€)', 'rating', 'has_5g', 'has_nfc',
       'has_ir_blaster', 'processor_brand', 'num_cores', 'processor_speed',
       'battery_capacity', 'fast_charging_available', 'fast_charging',
       'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate',
       'num_rear_cameras', 'num_front_cameras', 'os', 'primary_camera_rear',
       'primary_camera_front', 'extended_memory_available', 'extended_upto',
       'resolution_width', 'resolution_height'],
      dtype='object')

Maintenant, nous allons passer par l'étape d'exploration des données.

### EDA

### Importation des modules
import plotly.express as px
import plotly.graph_objects as go

**Colonne des marques**

### Distribution des marques
fig_brand = px.histogram(data, x='brand_name', nbins=20, title="Distribution des marques de téléphones vendus")

fig_brand.update_layout(
    xaxis_title='Marque',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_brand.show()

* Marques les plus fréquentes :
    - Xiaomi, OnePlus, et Samsung sont les plus représentées, avec environ 130 modèles ou variantes chacun.
    - Apple, Realme, Motorola, Infinix, Vivo, Oppo et Nokia suivent avec une forte présence.
* Longue traîne de petites marques :
    - De nombreuses marques (plus de 20) sont représentées de façon marginale (moins de 10 modèles), voire parfois très peu présentes.


**Colonnes des prix**

Ici, nous affichons la distribution des prix des smartphones.

In [6]:
### Distribution des prix
fig_price = px.histogram(data, x='price (€)', nbins=20, title='Distribution des prix des smartphones')

fig_price.update_layout(
    xaxis_title='Prix (€)',
    yaxis_title='Nombre de smartphones',
    bargap=0.1
)

fig_price.show()

* Distribution très asymétrique : la majorité écrasante des smartphones sont concentrés dans la tranche basse des prix, en dessous de 1 000 €.
* Longue queue vers la droite : quelques smartphones très haut de gamme (jusqu’à 6000 €) tirent la distribution vers la droite, mais ils sont extrêmement rares.
* Pointe forte autour de l'entrée de gamme : le pic de fréquence se situe dans les premiers intervalles de prix (probablement entre 100 € et 500 €).

**Prix vs. Capacité de la batterie**

In [7]:
fig_battery_price = px.scatter(
    data,
    x='battery_capacity',
    y='price (€)',
    color='brand_name',
    title='Prix vs Capacité de batterie des smartphones',
    labels={'battery': 'Batterie (mAh)', 'price': 'Prix (€)'},
    opacity=0.6
)

fig_battery_price.update_layout(
    xaxis_title='Capacité batterie',
    yaxis_title='Prix (€)',
    bargap=0.1
)

fig_battery_price.show()

* Tendance générale :
    - Il n’y a pas de corrélation claire entre le prix et la capacité de batterie.
    - Des téléphones à prix très bas peuvent avoir une batterie très élevée (outliers visibles à droite).
    - Inversement, certains téléphones chers n'ont qu'une capacité moyenne.
* Présence d’outliers :
    - Des smartphones affichent une capacité batterie supérieure à 10 000 mAh, ce qui est anormalement élevé pour des téléphones classiques → probablement des erreurs ou des modèles atypiques (tablettes, rugged phones).
* Dispersion importante :
    - Il y a une forte variabilité des prix pour une même capacité batterie, ce qui montre que la batterie n’est pas un facteur déterminant du prix.

**Comparaison entre le prix et les différentes colonnes**

In [8]:
### Liste des colonnes à comparer au prix
colonnes = ['rating', 'num_cores', 'processor_speed', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', ]

### Création des traces pour chaque colonne
fig_compare_to_price = go.Figure()

for i, col in enumerate(colonnes):
    fig_compare_to_price.add_trace(
        go.Scatter(
            x=data[col],
            y=data['price (€)'],
            mode='markers',
            name=col,
            visible=(i == 0)
        )
    )

### Création du menu déroulant
menu = [dict(
    buttons=[dict(
        label=col,
        method='update',
        args=[{'visible': [i == j for j in range(len(colonnes))]},
              {'xaxis': {'title': col}}]
    ) for i, col in enumerate(colonnes)],
    direction='down',
    showactive=True,
    x=0.1,
    y=1.15
)]

fig_compare_to_price.update_layout(
    updatemenus=menu,
    title='Prix vs variable sélectionnée',
    xaxis_title=colonnes[0],
    yaxis_title='Prix (€)',
    height=600
)

fig_compare_to_price.show()

**Rating vs prix**

* On observe une légère tendance croissante : les téléphones avec un meilleur rating tendent à être plus chers, surtout à partir de 75+.
* Cela peut s’expliquer par le fait que les modèles mieux notés sont généralement plus performants ou mieux finis.
* La majorité des téléphones ont une note comprise entre 75 et 90.
* Très peu de modèles en dessous de 70, ce qui peut indiquer un biais dans les avis (seuls les bons modèles sont évalués ou conservés dans le dataset).

**Processor_speed vs prix**

* On observe une corrélation positive modérée : en général, plus la vitesse du processeur augmente, plus les smartphones ont tendance à être chers.
* Beaucoup de modèles se situent entre 2.0 et 3.2 GHz, avec des prix allant de 100 € à 1000 €.
* Cela semble représenter le segment "standard à performant" du marché.
* Plus on monte en vitesse, plus les prix deviennent variables, indiquant que la vitesse du processeur n’est pas le seul facteur déterminant du prix.

**Internal_memory vs prix**

* Il existe une relation positive globale entre la capacité de stockage (en Go) et le prix du smartphone.
* On voit des groupes bien distincts :
    - 32–64 Go : téléphones d’entrée de gamme à prix très accessibles.
    - 128–256 Go : majorité des modèles milieu/haut de gamme.
    - 512–1024 Go (1 To) : modèles premium uniquement.
* À 256 Go, par exemple, les prix vont de ~400 € à plus de 2000 € → cela indique que le stockage ne suffit pas à expliquer le prix à lui seul.
* D’autres caractéristiques comme la RAM, le processeur, la marque ou la caméra jouent un rôle majeur.

**Screen_size vs prix**

* Il existe une légère corrélation positive entre la taille de l’écran et le prix : les téléphones avec un écran plus grand sont en moyenne un peu plus chers.
* Très forte densité autour de 6.3 à 6.7 pouces, avec des prix allant de 100 € à 1500 €.
* Cela suggère que la plupart des smartphones sont conçus pour offrir un compromis entre confort d’écran et ergonomie.
* À taille d’écran égale, le prix peut énormément varier → ce qui confirme que l’écran est un critère secondaire dans la tarification par rapport à :
    - la marque,
    - le processeur,
    - les fonctionnalités avancées (caméras, design, etc.).

**Refresh_rate vs prix**

* On observe une élévation du prix moyen à mesure que le taux de rafraîchissement augmente.
    - 60 Hz → présent sur des modèles de toutes gammes (entrée à premium).
    - 120 Hz et + → associés à des smartphones plus récents ou gaming, donc souvent plus chers.
* À 60 Hz, très grande dispersion de prix (de < 100 € à > 6000 €), ce qui confirme que ce n’est pas un critère déterminant seul.
* À 120–144 Hz, les prix restent modérément élevés avec moins de dispersion : cette technologie est réservée aux gammes moyennes à hautes.
* À 240 Hz, très peu de modèles → probablement des modèles gaming haut de gamme.

---

## Preprocessing des données

Nous allons maintenant procéder à un pré-traitement approfondi des données issues du dataset de smartphones disponible sur Kaggle. L’objectif est de nettoyer, transformer et enrichir ces données afin de construire un jeu de données final, adapté à la conception d’un modèle d’intelligence artificielle dédié à la recommandation de smartphones.

**Suppression de colonnes**

Dans cette section, nous allons supprimer certaines colonnes qui ne nous semblent pas pertinentes, ou déterminantes pour le choix d'un téléphone, notamment des fonctionnalités qui sont gadgets comme `has_ir_blaster`. Cela permettra d'être plus orienté vers la satisfaction de la clientèle.

In [9]:
### Suppression des colonnes
data = data.drop(columns=['has_ir_blaster', 'has_nfc'])

**Critère d'évaluation de l'autonomie d'un téléphone**

Maintenant, nous allons rajouter une étiquette à chaque téléphone pour évaluer le niveau d'autonomie de chaque téléphone, dans une démarche de satisfaction client. Si un client souhaite un téléphone avec une bonne autonomie, et rien d'autre cela permettra de mieux l'orienter vers ce qu'il recherche.

In [10]:
### Fonction d'étiquetage des téléphones par rapport à leur autonomie
def evaluate_battery_capacity(capacity) -> str:
    """
    Attribue une étiquette à chaque téléphone sur le seul critère de la capacité de leur batterie.
    
    :param capacity: Capacité du téléphone
    :return: Étiquette finale
    """
    
    if capacity < 3500:
        return "faible"
    elif capacity < 4500:
        return "moyenne"
    else:
        return "haute"

In [11]:
### Application de la fonction
data['quality_battery_autonomy'] = data['battery_capacity'].apply(evaluate_battery_capacity)

**Création d'une fonction de scoring pour catégoriser les téléphones selon leur performance**

Dans la section suivante, nous allons implémenter une fonction dans laquelle chaque ligne du dataset sera analysée, et pour chaque attribut des téléphones, nous allons attribuer un score qui sera utilisé après pour calculer un score final qui déterminera la catégorie du téléphone. Cela permettra d'affiner les futures recommandations du modèle, et de fournir des résultats qui pourront répondre aux besoins du client.

In [12]:
### Importation module
from typing import List, Tuple

### Fonction qui calcule un score total de chaque téléphone
def compute_total_score(phone) -> float:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haute performance, performance intermédiaire ou basique.
    :param phone: Correspond au téléphone (ligne) que l'on considère actuellement pour le calcul du score
    :return: Le score final du téléphone en fonction des différents critères d'évaluation
    """
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes numériques
    def score_range(val, seuils: List[Tuple[float, float]]) -> float:
        for seuil, point in seuils:
            if val >= seuil:
                return point
        return 0
    
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes booléennes
    def score_boolean(val, point_if_true=0.5) -> float:
        return point_if_true if val else 0
    
    ### Variable qui stocke le score final
    score: float = 0
    
    ### Poids de chaque caractéristique des téléphones attribués selon l'importance de la caractéristique pour le choix du téléphone
    weights: List[float] = [0.10, 0.10, 0.20, 0.10, 0.15, 0.10, 0.10, 0.10, 0.05]
    
    ### Calcul des scores : colonnes numériques
    score += score_range(phone['rating'], [(85, 1), (80, 0.5)]) * weights[0]
    score += score_range(phone['num_cores'], [(8, 1), (6, 0.5)]) * weights[1]
    score += score_range(phone['processor_speed'], [(2.8, 1), (2.4, 0.5)]) * weights[2]
    score += score_range(phone['battery_capacity'], [(4500, 1), (3500, 0.5)]) * weights[3]
    score += score_range(phone['ram_capacity'], [(12, 1), (8, 0.5)]) * weights[4]
    score += score_range(phone['internal_memory'], [(256, 1), (128, 0.5)]) * weights[5]
    score += score_range(phone['refresh_rate'], [(120, 1), (90, 0.5)]) * weights[6]
    score += score_range(phone['primary_camera_rear'], [(64, 1), (48, 0.5)]) * weights[7]
    
    ### Calcul des scores : colonnes booléennes
    score += score_boolean(phone['has_5g'], 1) * weights[-1]
    
    return score
        

Nous pouvons voir également que la colonne `rating` possède des valeurs manquantes qui sont cruciales pour le calcul du score de chaque téléphone. Pour remplacer ces valeurs, nous allons calculer les valeurs médianes des `rating` par marque de téléphones, et les attribuer aux téléphones qui n'ont pas de note répertoriée.

In [13]:
### Calcul des médianes des `rating` par marque
median_per_brand = data.groupby('brand_name')['rating'].median()

data['rating'] = data.apply(
    lambda phone: median_per_brand[phone['brand_name']] if pd.isna(phone['rating']) else phone['rating'],
    axis=1
)

In [14]:
### Fonction de catégorisation des téléphones
def determine_performance(phone) -> str:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haut de gamme, milieu de gamme ou bas de gamme.
     
    :param phone: Prend en argument chaque téléphone (ligne) du dataset
    :return: Une chaîne de caractères qui indique la catégorie du téléphone
    """
    score: float = compute_total_score(phone)
    
    ### Attribution des catégories
    if score >= 0.7:
        return "haut performance"
    elif score >= 0.4:
        return "performance intermédiaire"
    else:
        return "basique"

In [15]:
### Application au dataset
data['performance category'] = data.apply(determine_performance, axis=1)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,1.0,ios,12.0,12.0,0,NaN,828,1792,faible,basique
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,1.0,ios,12.0,12.0,0,NaN,828,1792,faible,basique
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,1.0,ios,12.0,12.0,0,NaN,1242,2688,moyenne,basique
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,1.0,ios,12.0,12.0,0,NaN,1170,2532,faible,basique
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,1.0,ios,12.0,12.0,0,NaN,1170,2532,faible,basique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,48.0,16.0,1,512.0,1080,2400,haute,basique
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,48.0,16.0,1,512.0,1080,2400,haute,basique
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,64.0,32.0,1,512.0,1080,2400,haute,basique
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,1.0,android,50.0,16.0,1,NaN,1080,2460,moyenne,performance intermédiaire


**Estimation du rapport qualité/prix**

Pour prendre en compte l'avis des utilisateurs sur les téléphones par rapport au prix de vente qu'ils ont, nous allons créer une nouvelle métrique `quality-price ratio`. Nous avons vu que pour une même note, par exemple 85, les prix peuvent varier de moins de `100 €` à plus de `1000 €`, ce qui montre que la note ne reflète pas uniquement le prix, mais elle peut aussi intégrer la qualité perçue, la marque, la fiabilité, ou encore l’expérience utilisateur.

Cependant, pour définir cette métrique, nous allons l'appliquer par segment de gamme. C'est-à-dire qu'un téléphone de haute gamme peut avoir, avec cette nouvelle métrique, un ratio qui peut être bas. Et, nous devons noter que, les téléphones haut de gamme ne sont pas destinés à être économique, il n'est pas pertinent d'estimer ce rapport pour cette gamme de téléphone. Donc, pour ce faire, nous allons créer des ratios par gamme. 

Alors, nous allons utiliser la proposition suivante de segmentation des prix :
- Haut de gamme : > 700 €
- Milieu de gamme : 300 - 700 €
- Entrée de gamme : < 300 €

Nous allons créer une nouvelle colonne qui recense le segment du prix auquel appartient chaque téléphone.

In [16]:
### Fonction de segmentation des prix
def segmenting_prices(price : float) -> str:
    """
    Détermine le segment dans lequel un téléphone se trouve à partir de son prix.
    
    :param price: Prix du téléphone
    :return: Segment auquel il appartient
    """
    if price < 300:
        return "bas"
    elif price <= 700:
        return "milieu"
    else:
        return "haut"

In [17]:
### Application de la fonction de segmentation
data['price_segment'] = data['price (€)'].apply(segmenting_prices)

Nous allons maintenant calculer le ratio `rating / price`, mais étant donné qu'il n'est pas comparable entre les différents segments, nous allons ensuite normaliser le ratio dans chaque segment. Cela donne un score de 0 à 1, relatif à la performance du téléphone dans sa gamme de prix. Cela nous permettra d'avoir des choses comme, un téléphone à 200 € avec 80 de rating et un à 1000 € avec 90 de rating ayant des scores similaires dans leur segment respectif, car chacun est bon dans sa catégorie.

In [18]:
### Calcul du ratio qualité / prix
data['ratio_rating_price'] = data['rating'] / data['price (€)']

In [19]:
### Normalisation des ratios dans chaque segment de prix
data['ratio_segment'] = data.groupby('price_segment')['ratio_rating_price'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category,price_segment,ratio_rating_price,ratio_segment
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,12.0,0,NaN,828,1792,faible,basique,milieu,0.187825,0.479802
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,12.0,0,NaN,828,1792,faible,basique,milieu,0.160123,0.327847
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,12.0,0,NaN,1242,2688,moyenne,basique,haut,0.070303,0.528967
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,12.0,0,NaN,1170,2532,faible,basique,milieu,0.142796,0.232805
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,12.0,0,NaN,1170,2532,faible,basique,milieu,0.134387,0.186675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,16.0,1,512.0,1080,2400,haute,basique,bas,0.537596,0.208098
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,16.0,1,512.0,1080,2400,haute,basique,bas,0.535094,0.206536
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,32.0,1,512.0,1080,2400,haute,basique,bas,0.486796,0.176399
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,16.0,1,NaN,1080,2460,moyenne,performance intermédiaire,bas,0.411419,0.129366


Voici une petite analyse de la distribution du ratio segmenté :

* Segment bas (< 300 €) :
    * `Moyenne : 0.21`
    * `25% des modèles ont un score < 0.11`
    * `Score élevé (75e percentile) : > 0.27`
    * Les bons scores sont rares dans ce segment, les valeurs sont globalement faibles.

* Segment milieu (300–700 €) :
    * `Moyenne : 0.51`
    * `25% des modèles : < 0.28`
    * `75% des modèles : < 0.75`
    * Cette distribution est plus équilibrée, nous avons de bons segments pour identifier des rapports qualités-prix corrects.

* Segment haut (> 700 €) :
    * `Moyenne : 0.61`
    * `25% des modèles : < 0.46`
    * `75% des modèles : > 0.77`
    * Ici, nous avons plus de variance, et certains modèles haut de gamme sont bien notés pour leur prix.

Nous allons donc écrire une fonction nous permettant d'attribuer nos étiquettes `qualité/prix` selon des seuils basés sur l'analyse des distributions des différents segments.

In [20]:
### Fonction d'étiquetage
def label_ratio(phone) -> str:
    """
    Fonction d'attribution de label sur le rapport qualité/prix en fonction du segment dans lequel se trouve le téléphone considéré.
    
    :param phone: Téléphone à étiqueter sur son rapport qualité/prix
    :return: Étiquette finale
    """
    
    ### Récupération des ratios
    val = phone['ratio_segment']
    segment = phone['price_segment']
    
    ### Disjonction de cas et attribution des labels
    ### Segment bas
    if segment == 'bas':
        if val >= 0.30:
            return "excellent"
        elif val >= 0.20:
            return "bon"
        elif val >= 0.10:
            return "moyen"
        else:
            return "faible"
        
    ### Segment milieu
    elif segment == 'milieu':
        if val >= 0.70:
            return "excellent"
        elif val >= 0.50:
            return "bon"
        elif val >= 0.30:
            return "moyen"
        else:
            return "faible"
    
    ### Segment haut
    elif segment == 'haut':
        if val >= 0.80:
            return "excellent"
        elif val >= 0.60:
            return "bon"
        elif val >= 0.40:
            return "moyen"
        else:
            return "faible"
    
    ### Renvoyer inconnu par mesure de sécurité (Possibles valeurs manquantes qui ne peuvent pas être remplacées)
    return "inconnu"

In [21]:
### Application de la fonction d'étiquetage
data['quality-price_ratio'] = data.apply(label_ratio, axis=1)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category,price_segment,ratio_rating_price,ratio_segment,quality-price_ratio
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,0,NaN,828,1792,faible,basique,milieu,0.187825,0.479802,moyen
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,0,NaN,828,1792,faible,basique,milieu,0.160123,0.327847,moyen
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,0,NaN,1242,2688,moyenne,basique,haut,0.070303,0.528967,moyen
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,0,NaN,1170,2532,faible,basique,milieu,0.142796,0.232805,faible
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,0,NaN,1170,2532,faible,basique,milieu,0.134387,0.186675,faible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.537596,0.208098,bon
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.535094,0.206536,bon
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.486796,0.176399,moyen
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,1,NaN,1080,2460,moyenne,performance intermédiaire,bas,0.411419,0.129366,moyen


**Enregistrement du dataframe**

Dans un premier temps, nous allons réorganiser les colonnes, et supprimer certaines dont nous n'aurons pas besoin pour la mise en place de notre modèle.

In [22]:
### Nouvel ordre des colonnes
new_order = ['brand_name', 'model', 'price (€)', 'price_segment','rating', 'ratio_rating_price', 'ratio_segment', 'quality-price_ratio', 'has_5g', 'processor_brand', 'num_cores', 'processor_speed', 'battery_capacity', 'quality_battery_autonomy', 'fast_charging_available', 'fast_charging', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', 'num_rear_cameras', 'num_front_cameras', 'os', 'primary_camera_rear', 'primary_camera_front', 'extended_memory_available', 'extended_upto', 'resolution_width', 'resolution_height', 'performance category']

data = data[new_order]

**Extraction du dataset traîté**

In [23]:
data.to_csv('../data/processed/Smartphones_cleaned_dataset_processed.csv', index=False)

---
## Pipeline RAG

1. Chargement des données et configuration  
2. Construction du jeu de passages + embeddings DPR  
3. Création de l’index FAISS  
4. Chargement du pipeline RAG Hugging-Face  
5. Tests d’interrogation  
6. Interprétation des résultats

In [24]:
### Importation des modules
import os
import re
import faiss
import json
import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk, load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast
)
from sentence_transformers import SentenceTransformer
from peft import PeftModel
from typing import *

/opt/anaconda3/envs/phone-recommender/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning:

Unable to find acceptable character detection dependency (chardet or charset_normalizer).



In [25]:
### Séparation de l'import de ce module car il génère des erreurs dans l'environnement
import torch

In [26]:
### Chargement des variables d'environnement
TRANSFORMER = os.getenv('TRANSFORMER')
MODEL_NAME = os.getenv("MODEL_NAME")
GEN_MODEL = os.getenv("GEN_MODEL")
PASSAGES_PATH = os.getenv("PASSAGES_PATH")
DPR_DATASET = os.getenv("DPR_DATASET")
INDEX_PATH = os.getenv("INDEX_PATH")
N_DOCS = int(os.getenv("N_DOCS", 50))
DEVICE = os.getenv("DEVICE")

### 1. Création des passages

Ici, nous allons procéder à la création des passages. On lit le CSV nettoyé et on génère un fichier JSONL de `{"title","text"}`, où `title` est le nom de la marque, et le modèle du téléphone, et `text` correspond à toutes les caractéristiques d'un téléphone : c'est-à-dire le prix, la taille d'écran, etc.

L'idée est de transformer les informatiosn en un language semi-naturel pour aiser la compréhension du modèle des informations. Effectivement, les bien qu'une structure *caractéristique : valeur* est compréhensible à nos yeux, il est plus difficile à interpréter pour FAISS.

In [27]:
records: list = []
for _, phone in data.iterrows():
    title = f"{phone['model']}"
    
    ### Formatage du texte
    text = (
        f"Le {phone['brand_name']} {phone['model']} coûte {phone['price (€)']} €, ce qui en fait un modèle avec un prix {phone['price_segment']}. "
        f"Il dispose d'un écran de {phone['screen_size']} pouces avec un taux de rafraîchissement de {phone['refresh_rate']} Hz. "
        f"Ce télépone a une note de {phone['rating']}, un rapport qualité-prix de {phone['quality-price_ratio']} et {'dispose' if phone['has_5g'] else 'ne dispose pas'} de la 5G. "
        f"Ce téléphone offre une autonomie {phone['quality_battery_autonomy']} grâce à une batterie de {phone['battery_capacity']} mAh. "
        f"Il est équipé de {phone['internal_memory']} Go de stockage, de {phone['ram_capacity']} Go de RAM ce qui en fait un téléphone avec un niveau de performance {phone['performance category']}. "
        f"Il tourne sous {phone['os']}, possède une caméra arrière de {phone['primary_camera_rear']} Mpx et une caméra avant de {phone['primary_camera_front']} Mpx. "
    )
    records.append({"title": title, "text": text})

### Enregistrement des passages dans un dataset
with open('../data/processed/passages.jsonl', 'w') as f:
    for passage in records:
        f.write(json.dumps(passage, ensure_ascii=False) + "\n")

print("Passages enregistrés dans", "../data/processed/passages.jsonl")

Passages enregistrés dans ../data/processed/passages.jsonl


### 2. Création du dataset et embeddings

Ici, il s'agira d'encoder chaque `text` avec Sentence-Transformer et d'ajouter la colonne `embeddings`.

In [28]:
### Chargement des passages dans un dataset
ds = Dataset.from_json("../data/processed/passages.jsonl", split="train")

### Récupération des phrases dans les passages
texts: list = list(ds['text'])

"""
Création d'un embedder pour le calcul des embeddings
DEVICE définit quel matériel (Hardware ici) sur lequel nous faisons les calculs des embeddings : CPU ou GPU
Transformer : all-MiniLM-L6-v2
"""
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=DEVICE)

### Calcul des embeddings
embs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)

### Ajout de la colonne `embeddings`
ds = ds.add_column("embeddings", embs.tolist())

### Sauvegarde du dataset
ds.save_to_disk("../data/processed/passages_dataset_dpr")
print("Dataset DPR sauvé dans", "../data/processed/passages_dataset_dpr")

Generating train split: 0 examples [00:00, ? examples/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/979 [00:00<?, ? examples/s]

Dataset DPR sauvé dans ../data/processed/passages_dataset_dpr


**Affichage embeddings**

In [29]:
### Chargement des embeddings
embeddings = np.array(ds["embeddings"], dtype=np.float32)

### On vérifie ici la dimension des embeddings
print("Dimension des embeddings:", embeddings.shape)

### Affichage des 5 premiers embeddings
print("5 premiers embeddings :", embeddings[:5])

Dimension des embeddings: (979, 384)
5 premiers embeddings : [[-0.05384278  0.13649024 -0.01815253 ... -0.00907882 -0.02675149
   0.01626959]
 [-0.04387905  0.1302784  -0.01567334 ... -0.01041287 -0.03648336
   0.01302907]
 [-0.05503476  0.11413612 -0.02420181 ... -0.00901012 -0.01617312
   0.01406909]
 [-0.02209838  0.13988084 -0.02962287 ... -0.00617501 -0.01457006
   0.03625868]
 [-0.0154454   0.13735439 -0.02989352 ... -0.00852892 -0.02043919
   0.03482395]]


### 3. Création de l’index FAISS

Ici, nous chargeons le dataset DPR, puis extraction des embeddings et on bâtit un `IndexFlatIP`.

In [30]:
### Chargement des embeddings
ds2 = load_from_disk("../data/processed/passages_dataset_dpr")

emb_np = np.array(ds2["embeddings"], dtype=np.float32)

### Création de l'index
index = faiss.IndexFlatIP(emb_np.shape[1])

index.add(emb_np)

faiss.write_index(index, "../data/processed/faiss_index.idx")

print("Index FAISS enregistré dans", "../data/processed/faiss_index.idx")

Index FAISS enregistré dans ../data/processed/faiss_index.idx


### 4. Fonction de retrieval

Ici, nous allons encoder la question poser par l'utilisateur, et effectuer la recherche dans FAISS.

In [31]:
### Chargement de l'index
faiss_idx = faiss.read_index("../data/processed/faiss_index.idx")
passages = ds2["text"]

In [32]:
### Fonction retriever
def retrieve(q: str, top_k: int = 10):
    """
    Rechercher les top_k passages les plus pertinents pour une question donnée.

    :param q: Question ou requête à rechercher.
    :param top_k: Nombre de résultats à retourner. Par défaut 10.

    :return list of tuples: Liste de tuples (passage, score) des passages les plus pertinents.
    """
    ### Encodage de la question - Utilisation du DEVICE
    q_emb = embedder.encode([q], convert_to_numpy=True)

    ### Recherche FAISS
    ### scores, ids = faiss_idx.search(rep.astype(np.float32), k=top_k)
    scores, ids = faiss_idx.search(q_emb.astype("float32"), top_k)
    
    ### Conversion explicite des indices en entier
    docs = [(passages[int(i)], float(scores[0, idx])) for idx, i in enumerate(ids[0])]

    return docs

**Tests du FAISS**

In [33]:
# Question test
question = "Je veux un smartphone avec une batterie de 5000 mAh"

question_embedding = embedder.encode([question], convert_to_numpy=True)
scores, ids = faiss_idx.search(question_embedding.astype(np.float32), k=10)
for i, identifiant in enumerate(ids[0]):
    doc_id = int(identifiant)
    print(f"Doc {i+1}: {passages[doc_id]} (Score: {scores[0][i]})")

Doc 1: Le realme Realme Narzo 50A coûte 114.6 €, ce qui en fait un modèle avec un prix bas. Il dispose d'un écran de 6.5 pouces avec un taux de rafraîchissement de 60 Hz. Ce télépone a une note de 71.0, un rapport qualité-prix de bon et ne dispose pas de la 5G. Ce téléphone offre une autonomie haute grâce à une batterie de 6000 mAh. Il est équipé de 64 Go de stockage, de 4 Go de RAM ce qui en fait un téléphone avec un niveau de performance basique. Il tourne sous android, possède une caméra arrière de 50.0 Mpx et une caméra avant de 8.0 Mpx.  (Score: 0.7045806646347046)
Doc 2: Le samsung Samsung Galaxy A50s coûte 159.36 €, ce qui en fait un modèle avec un prix bas. Il dispose d'un écran de 6.4 pouces avec un taux de rafraîchissement de 60 Hz. Ce télépone a une note de 80.0, un rapport qualité-prix de moyen et ne dispose pas de la 5G. Ce téléphone offre une autonomie moyenne grâce à une batterie de 4000 mAh. Il est équipé de 128 Go de stockage, de 4 Go de RAM ce qui en fait un téléphone

On peut ici voir que le FAISS est parvenu à extraire un contexte de téléphones ayant une autonomie de batterie de 5000 mAh. Cependant, on peut tout de même voir qu'il y a quelques erreurs avec des téléphones à plus petite autonomie. 

### 5. Création du modèle avec Google flan-T5

Dans notre cas d'usage, nous avons choisi le modèle de google de FLAN-T5 car c'est un modèle encodeur-décodeur basé sur les transformers, affiné grâce à l’instruction tuning. Cela signifie qu’il est entraîné pour comprendre des consignes en langage naturel (questions des ulitisateurs) et générer des réponses cohérentes. 

Ce modèle est particulièrement adapté aux systèmes de question-réponse liée à un RAG par récupération de documents, où il doit lire un contexte précis puis formuler une réponse justifiée. Par rapport à d'autres modèles comme GPT qui sont décodeur-only, le FLAN-T5 gère mieux les tâches où la réponse doit être directement liée à un contexte fourni, ce qui est exactement le cas de notre problème (recommandation de téléphones).

**Partie génération Seq2Seq**

In [34]:
### Mise en place du modèle de base Google flan-T5
gen_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
gen_mod = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base",
    torch_dtype=torch.float16,
).to(DEVICE)

### Setup de la taille maximale du tokenizer
MAX_PROMPT_LEN = 1024 or 512

In [35]:
def ask(question: str) -> str:
    ### Obtention du contexte calculé par FAISS plus tôt
    docs = retrieve(question, top_k=3)
    context = "\n\n".join(f"[Doc {i+1}] {txt}" for i,(txt,_) in enumerate(docs))

    
    ### Ajout d'exemples pour que le modèle puisse répondre par des phrases structurées et argumentées aux questions posées par l'utilisateur
    few_shot = """Tu es un assistant expert. En te basant uniquement sur le contexte fourni, réponds à la question en comparant les téléphones et en faisant une recommandation claire et justifiée.

Exemple 1 :
Question : Je cherche un téléphone avec une bonne autonomie et un prix milieu.
Contexte :
- Téléphone A : Batterie 5000 mAh, Prix 250 €, Note 74
- Téléphone B : Batterie 3110 mAh, Prix 240 €, Note 72
Réponse : Le téléphone A est plus adapté grâce à sa batterie de 5000 mAh, tout en restant dans la gamme de prix souhaitée. Il offre un meilleur équilibre entre autonomie et prix.

""".strip()

    ### Création d'un pour le modèle qui contient l'exemple, le contexte et la question
    prompt = few_shot + "\n" + \
             f"Vrai contexte à utiliser :\n{context}\n\n" + \
             f"Vraie question à laquelle répondre : {question}\nRéponse :"

    inputs = gen_tok(prompt, return_tensors="pt", truncation=True,  max_length=2048).to(DEVICE)
    
    inputs.pop("token_type_ids", None)

    ### Génération du résultat
    out = gen_mod.generate(
        **inputs,
        max_new_tokens=650,
        num_beams=8,
        no_repeat_ngram_size=4,
        length_penalty=1.2,
        early_stopping=True
    )
    return gen_tok.decode(out[0], skip_special_tokens=True)

Exemple d'utilisation du modèle en posant une question ulitisateur

In [36]:
### Test de la génération
q = ("Je veux un smartphone avec un prix bas")
print("Q:", q)
print("A:", ask(q), "\n")

### Affichage des scores FAISS
for rank, (doc, score) in enumerate(retrieve(q, top_k=5), start=1):
    print(f"{rank:2d}. (score={score:.4f}) {doc}")

Attempting to cast a BatchEncoding to type None. This is not supported.


Q: Je veux un smartphone avec un prix bas
A: [Doc 1] Le realme Realme Narzo 50i coûte 64.68 €, ce qui en fait un modèle avec un prix bas. Il dispose d'un écran de 6.5 pouches avec un taux de rafraîchissement de 60 Hz. Ce télépone a une note de 78.5, un rapport qualité-prix excellent et ne dispose pas de 5G. Ce téléphone offre une autonomie haute grâce à une batterie de 5000 mAh. Il est équipée de 32 Go stockage, de 2 Go RAM, ce qui fait un téléphone avec un niveau de performance basique. Il tourne sous android, possède une caméra arrière 13.0 Mpx et une caméra avant de 5.0 Mpx. [Doc 2] Le oneplus OnePlus Nord N200 coûte 169.32 €, cet énoncé modèle avec une prix bas. 

 1. (score=0.6826) Le realme Realme Narzo 50i coûte 64.68 €, ce qui en fait un modèle avec un prix bas. Il dispose d'un écran de 6.5 pouces avec un taux de rafraîchissement de 60 Hz. Ce télépone a une note de 78.5, un rapport qualité-prix de excellent et ne dispose pas de la 5G. Ce téléphone offre une autonomie haute grâc

### 6. Utilisation du modèle fine-tuné

Pour améliorer les performances de notre modèle de recommandation de smartphones, nous allons procéder à une phase de fine-tuning.

**Etape du fine-tuning**

Nous avons réaliser ce code dans un fichier python externe

**Chargement des modules**
```python
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
```

**Chargement des données Q&A**
```python
ds = load_dataset("json", data_files="../../data/external/qa_smartphones.jsonl")
```

**Chargement du tekonizer et du modèle de base**
```python
MODEL = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL)
```
**Configuration LoRA**
```python
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,           # rang des matrices LoRA
    lora_alpha=32,
    lora_dropout=0.05
)
model = get_peft_model(model, peft_config)
```
**Pré-traitement**
```python
def preprocess(ex):
    inp = ex["input_text"]
    tgt = ex["target_text"]
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(
        tgt, truncation=True, padding="max_length", max_length=128
    ).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized = ds["train"].map(preprocess, batched=True, remove_columns=ds["train"].column_names)
```
**Entraînement**
```python
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs/",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    num_train_epochs=3,
    fp16=False,
    logging_steps=100,
    save_total_limit=2,
    do_eval=False,
    optim="adamw_torch"
    # eval_steps=200,
    # evaluation_strategy="steps"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized
)

trainer.train()
model.save_pretrained("fine_tuned_flant5_smartphones")
tokenizer.save_pretrained("fine_tuned_flant5_smartphones")
```

In [37]:
### Chargement du modèle fine-tuné
BASE_MODEL = "google/flan-t5-base"
CHECKPOINT  = "../src/models/outputs/checkpoint-30"

In [38]:
### Chargement du tokenizer
tok = AutoTokenizer.from_pretrained(BASE_MODEL)

base = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
).to(DEVICE)

### Ajout du modèle fine tuné
model = PeftModel.from_pretrained(base, CHECKPOINT)

/opt/anaconda3/envs/phone-recommender/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning:

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.



'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [39]:
def ask_ft(question: str) -> str:
    docs = retrieve(question)
    context = "\n\n".join(f"[Doc {i+1}] {d}" for i,(d,_) in enumerate(docs))
    few_shot = (
        "Tu es un assistant expert en téléphonie mobile.\n"
        "Réponds en français de manière concise, claire et argumentée par rapport au contexte et à la question.\n\n"
    )

    prompt = (
        few_shot
      + "Contexte :\n" + context + "\n\n"
      + f"Question : {question}\n"
      + "Réponse :"
    )

    inputs = tok(prompt, return_tensors="pt", truncation=True).to(DEVICE)
    inputs.pop("token_type_ids", None)
    out = model.generate(
        **inputs,
        max_new_tokens=120,
        num_beams=4,
        no_repeat_ngram_size=3,
        length_penalty=1.2,
        early_stopping=True
    )
    return tok.decode(out[0], skip_special_tokens=True)

In [40]:
###
print( ask_ft("Je veux un téléphone avec une haute performance.") )

Attempting to cast a BatchEncoding to type None. This is not supported.


RuntimeError: Placeholder storage has not been allocated on MPS device!